In [8]:
import wandb
import pandas as pd

api = wandb.Api()
runs = api.runs(path='ASOCA_final', filters={
    'config.seed': {'$in': [0,11,42]},
    'config.model/model': {"$regex": "models.classification.*"}, 
    'createdAt': {'$gte': '2021-08-15' },
    'state': { '$ne': 'Running' },
})

In [9]:
res = []
for run in runs:
    if not run.config['dataset/sample_every_epoch']: continue
    if len(run.tags) == 1 and run.tags[0] == 'match_features': continue
    res.append([
        run.config['model/model'].split('.')[-1],
        run.config.get('model/repr_type', ''),
#         not run.config['dataset/sample_every_epoch'],
#         True if len(run.tags) == 1 and run.tags[0] == 'match_features' else False,
        run.summary.get('train/acc'), 
        run.summary.get('train/f1'),
        run.summary.get('valid/acc'), 
        run.summary.get('valid/f1'),
        run.summary.get('test/acc'), 
        run.summary.get('test/f1'),
    ])

In [10]:
results = pd.DataFrame(res, columns=['model', 'repr_type', #'match_feats', 'reduced_ds',
                           'train/acc', 'train/f1',
                           'valid/acc', 'valid/f1',
                           'test/acc', 'test/f1']).groupby(['model','repr_type']).agg(['mean', 'std'])

In [11]:
results

train/acc      train/f1      valid/acc  \
                                        mean  std     mean  std      mean   
model                    repr_type                                          
Baseline3DClassification                   1  0.0        1  0.0    0.9896   

                                              valid/f1            test/acc  \
                                         std      mean       std      mean   
model                    repr_type                                           
Baseline3DClassification            0.001587  0.988899  0.001445  0.985117   

                                               test/f1            
                                         std      mean       std  
model                    repr_type                                
Baseline3DClassification            0.001457  0.985075  0.001462

In [12]:
results_f = pd.DataFrame()

In [13]:
results_f['train/acc'] = results.apply(lambda row: f"{row['train/acc']['mean']:.3f} ({row['train/acc']['std']:.3f})", axis=1)
results_f['train/f1'] = results.apply(lambda row: f"{row['train/f1']['mean']:.3f} ({row['train/f1']['std']:.3f})", axis=1)
results_f['valid/acc'] = results.apply(lambda row: f"{row['valid/acc']['mean']:.3f} ({row['valid/acc']['std']:.3f})", axis=1)
results_f['valid/f1'] = results.apply(lambda row: f"{row['valid/f1']['mean']:.3f} ({row['valid/f1']['std']:.3f})", axis=1)
results_f['test/acc'] = results.apply(lambda row: f"{row['test/acc']['mean']:.3f} ({row['test/acc']['std']:.3f})", axis=1)
results_f['test/f1'] = results.apply(lambda row: f"{row['test/f1']['mean']:.3f} ({row['test/f1']['std']:.3f})", axis=1)

In [14]:
results_f

,,train/acc,train/f1,valid/acc,valid/f1,test/acc,test/f1
model,repr_type,,,,,,
Baseline3DClassification,,1.000 (0.000),1.000 (0.000),0.990 (0.002),0.989 (0.001),0.985 (0.001),0.985 (0.001)


In [15]:
print(results_f.to_latex(float_format="%.3f"))

\begin{tabular}{llllllll}
\toprule
                         &           &      train/acc &       train/f1 &      valid/acc &       valid/f1 &       test/acc &        test/f1 \\
model & repr\_type &                &                &                &                &                &                \\
\midrule
Baseline3DClassification &           &  1.000 (0.000) &  1.000 (0.000) &  0.990 (0.002) &  0.989 (0.001) &  0.985 (0.001) &  0.985 (0.001) \\
\bottomrule
\end{tabular}



In [20]:
res = []
for run in runs:
#     if any([
#         run.config['dataset/sample_every_epoch'],
#         run.config['dataset/sample_every_epoch'] and len(run.tags) == 0
#     ]): continue
    if run.config['model/model'].split('.')[-1] not in ['Baseline3DClassification', 'CubeCNN']: continue
    res.append([
        run.config['model/model'].split('.')[-1],
        True if len(run.tags) == 1 and run.tags[0] == 'match_features' else False,
        not run.config['dataset/sample_every_epoch'],
        run.summary.get('train/acc'), 
        run.summary.get('train/f1'),
        run.summary.get('valid/acc'), 
        run.summary.get('valid/f1'),
        run.summary.get('test/acc'), 
        run.summary.get('test/f1'),
    ])

In [21]:
results_abl = pd.DataFrame(res, columns=['model', 'match_feats', 'reduced_ds',
                           'train/acc', 'train/f1',
                           'valid/acc', 'valid/f1',
                           'test/acc', 'test/f1']).groupby(['model','match_feats', 'reduced_ds']).agg(['count', 'mean', 'std'])

In [26]:
results_abl

train/acc           train/f1  \
                                                    count mean  std    count   
model                    match_feats reduced_ds                                
Baseline3DClassification False       False              4    1  0.0        4   
                                     True               3    1  0.0        3   
CubeCNN                  False       False              4    1  0.0        4   
                                     True               3    1  0.0        3   

                                                          valid/acc            \
                                                mean  std     count      mean   
model                    match_feats reduced_ds                                 
Baseline3DClassification False       False         1  0.0         4  0.990500   
                                     True          1  0.0         3  0.977200   
CubeCNN                  False       False         1  0.0         4  0.990800   
                                     True          1  0.0         3  0.982533   

                                                          valid/f1            \
                                                      std    count      mean   
model                    match_feats reduced_ds                                
Baseline3DClassification False       False       0.001089        4  0.989950   
                                     True        0.005524        3  0.975572   
CubeCNN                  False       False       0.003154        4  0.990203   
                                     True        0.004120        3  0.981099   

                                                          test/acc            \
                                                      std    count      mean   
model                    match_feats reduced_ds                                
Baseline3DClassification False       False       0.001240        4  0.991437   
                                     True        0.006122        3  0.981967   
CubeCNN                  False       False       0.003681        4  0.990925   
                                     True        0.004166        2  0.984350   

                                                          test/f1            \
                                                      std   count      mean   
model                    match_feats reduced_ds                               
Baseline3DClassification False       False       0.000844       4  0.991424   
                                     True        0.003287       3  0.981908   
CubeCNN                  False       False       0.000674       4  0.990913   
                                     True        0.000212       2  0.984324   

                                                           
                                                      std  
model                    match_feats reduced_ds            
Baseline3DClassification False       False       0.000851  
                                     True        0.003254  
CubeCNN                  False       False       0.000676  
                                     True        0.000189

In [27]:
results_abl_f = pd.DataFrame()
results_abl_f['train/acc'] = results_abl.apply(lambda row: f"{row['train/acc']['mean']:.3f} ({row['train/acc']['std']:.3f})", axis=1)
results_abl_f['train/f1'] = results_abl.apply(lambda row: f"{row['train/f1']['mean']:.3f} ({row['train/f1']['std']:.3f})", axis=1)
results_abl_f['valid/acc'] = results_abl.apply(lambda row: f"{row['valid/acc']['mean']:.3f} ({row['valid/acc']['std']:.3f})", axis=1)
results_abl_f['valid/f1'] = results_abl.apply(lambda row: f"{row['valid/f1']['mean']:.3f} ({row['valid/f1']['std']:.3f})", axis=1)
results_abl_f['test/acc'] = results_abl.apply(lambda row: f"{row['test/acc']['mean']:.3f} ({row['test/acc']['std']:.3f})", axis=1)
results_abl_f['test/f1'] = results_abl.apply(lambda row: f"{row['test/f1']['mean']:.3f} ({row['test/f1']['std']:.3f})", axis=1)

In [28]:
results_abl_f

train/acc       train/f1  \
model                    match_feats reduced_ds                                 
Baseline3DClassification False       False       1.000 (0.000)  1.000 (0.000)   
                                     True        1.000 (0.000)  1.000 (0.000)   
CubeCNN                  False       False       1.000 (0.000)  1.000 (0.000)   
                                     True        1.000 (0.000)  1.000 (0.000)   

                                                     valid/acc       valid/f1  \
model                    match_feats reduced_ds                                 
Baseline3DClassification False       False       0.991 (0.001)  0.990 (0.001)   
                                     True        0.977 (0.006)  0.976 (0.006)   
CubeCNN                  False       False       0.991 (0.003)  0.990 (0.004)   
                                     True        0.983 (0.004)  0.981 (0.004)   

                                                      test/acc        test/f1  
model                    match_feats reduced_ds                                
Baseline3DClassification False       False       0.991 (0.001)  0.991 (0.001)  
                                     True        0.982 (0.003)  0.982 (0.003)  
CubeCNN                  False       False       0.991 (0.001)  0.991 (0.001)  
                                     True        0.984 (0.000)  0.984 (0.000)

In [30]:
print(results_abl_f.to_latex(float_format="%.3f"))

\begin{tabular}{lllllllll}
\toprule
        &       &       &      train/acc &       train/f1 &      valid/acc &       valid/f1 &       test/acc &        test/f1 \\
model & match\_feats & reduced\_ds &                &                &                &                &                &                \\
\midrule
Baseline3DClassification & False & False &  1.000 (0.000) &  1.000 (0.000) &  0.991 (0.001) &  0.990 (0.001) &  0.991 (0.001) &  0.991 (0.001) \\
        &       & True  &  1.000 (0.000) &  1.000 (0.000) &  0.977 (0.006) &  0.976 (0.006) &  0.982 (0.003) &  0.982 (0.003) \\
CubeCNN & False & False &  1.000 (0.000) &  1.000 (0.000) &  0.991 (0.003) &  0.990 (0.004) &  0.991 (0.001) &  0.991 (0.001) \\
        &       & True  &  1.000 (0.000) &  1.000 (0.000) &  0.983 (0.004) &  0.981 (0.004) &  0.984 (0.000) &  0.984 (0.000) \\
\bottomrule
\end{tabular}



In [59]:
res = []
for run in runs:
#     if any([
#         run.config['dataset/sample_every_epoch'],
#         run.config['dataset/sample_every_epoch'] and len(run.tags) == 0
#     ]): continue
    res.append([
        run.config['model/model'].split('.')[-1],
        run.config.get('model/repr_type', ''),
        run.summary.get('eq_max_error'), 
        run.summary.get('eq_mean_error'),
    ])

In [64]:
results_eq = pd.DataFrame(res, columns=['model', 'repr_type', 'max', 'mean']).groupby(['model', 'repr_type']).agg(['max', 'mean', 'std'])

In [68]:
results_eq_f = pd.DataFrame()
results_eq_f['max'] = results_eq.apply(lambda row: f"{row['max']['max']:.3f}", axis=1)
results_eq_f['mean'] = results_eq.apply(lambda row: f"{row['mean']['mean']:.3f} ({row['mean']['std']:.3f})", axis=1)

In [69]:
results_eq_f

max             mean
model                    repr_type                          
Baseline3DClassification             19.985    4.940 (2.857)
CubeCNN                              11.842    3.653 (1.861)
FTCNN                    so3         25.812    6.414 (5.411)
                         spherical    3.492    2.033 (0.416)
                         trivial      3.397    2.171 (0.230)
GatedCNN                 so3          3.492    1.974 (0.361)
                         spherical  103.069  15.039 (22.971)
                         trivial     18.776    3.627 (3.087)
IcoCNN                                8.849    3.515 (1.828)

In [70]:
print(results_eq_f.to_latex(float_format="%.3f"))

\begin{tabular}{llll}
\toprule
       &         &      max &             mean \\
model & repr\_type &          &                  \\
\midrule
Baseline3DClassification &         &   19.985 &    4.940 (2.857) \\
CubeCNN &         &   11.842 &    3.653 (1.861) \\
FTCNN & so3 &   25.812 &    6.414 (5.411) \\
       & spherical &    3.492 &    2.033 (0.416) \\
       & trivial &    3.397 &    2.171 (0.230) \\
GatedCNN & so3 &    3.492 &    1.974 (0.361) \\
       & spherical &  103.069 &  15.039 (22.971) \\
       & trivial &   18.776 &    3.627 (3.087) \\
IcoCNN &         &    8.849 &    3.515 (1.828) \\
\bottomrule
\end{tabular}



In [16]:
runs = api.runs(path='asoca_results')

In [24]:
for run in runs:
    model_type = run.config.get('train/model') or run.config.get('model/model').split('.')[-1]
    print(run.name, model_type)
    run.config.update({'model_type': model_type})
    run.update()

quiet-wind-777 FTUNet
driven-wind-776 FTUNet
feasible-plasma-775 FTUNet
easy-glade-774 CubeUNet
wobbly-durian-773 CubeUNet
dashing-sponge-772 CubeUNet
stellar-armadillo-775 UNet
bumbling-firefly-747 UNet
morning-leaf-745 UNet
effortless-smoke-621 IcoUNet
drawn-wildflower-617 IcoUNet
bumbling-galaxy-613 IcoUNet
prime-breeze-609 CubeUNet
electric-river-602 CubeUNet
deep-sky-601 CubeUNet
youthful-sound-363 cnn
balmy-glitter-356 cnn
bumbling-firebrand-355 cnn
devout-waterfall-344 unet
royal-firebrand-343 unet
lucky-frog-343 unet


In [103]:
run.history()['valid/hd95'].dropna()[-15:].mean()

18.780423768795867

In [104]:
import numpy as np

In [106]:
res = []
for run in runs:

    res.append([
        run.config.get('train/model') or run.config.get('model/model').split('.')[-1],
        run.history()['train/dice'].dropna()[-15:].mean(),
        run.history()['valid/dice'].dropna()[-15:].mean(),
        run.history()['valid/hd95'].apply(lambda x: x if x != 'Infinity' else np.nan).dropna()[-15:].mean(),
    ])

In [107]:
results_seg = pd.DataFrame(res, columns=['model', 'train/dice', 'valid/dice', 'valid/hd95']).groupby(['model']).agg(['mean', 'std'])


In [108]:

results_seg

train/dice           valid/dice           valid/hd95          
               mean       std       mean       std       mean       std
model                                                                  
CubeUNet   0.360264  0.352590   0.171164  0.212902  38.632774  5.026907
FTUNet     0.421951  0.311734   0.205797  0.178423  38.265541  2.108521
IcoUNet    0.154954  0.110115   0.035020  0.022373  40.030372  0.025534
UNet       0.532857  0.037367   0.249654  0.046438  37.094731  0.100324
cnn        0.585589  0.020756   0.456776  0.021958  36.650351  0.141176
unet       0.696163  0.035687   0.783633  0.023337  19.761207  0.721324

In [109]:
results_seg_f = pd.DataFrame()
results_seg_f['train/dice'] = results_seg.apply(lambda row: f"{row['train/dice']['mean']:.3f} ({row['train/dice']['std']:.3f})", axis=1)
results_seg_f['valid/dice'] = results_seg.apply(lambda row: f"{row['valid/dice']['mean']:.3f} ({row['valid/dice']['std']:.3f})", axis=1)
results_seg_f['valid/hd95'] = results_seg.apply(lambda row: f"{row['valid/hd95']['mean']:.3f} ({row['valid/hd95']['std']:.3f})", axis=1)

In [110]:
results_seg_f

,train/dice,valid/dice,valid/hd95
model,,,
CubeUNet,0.360 (0.353),0.171 (0.213),38.633 (5.027)
FTUNet,0.422 (0.312),0.206 (0.178),38.266 (2.109)
IcoUNet,0.155 (0.110),0.035 (0.022),40.030 (0.026)
UNet,0.533 (0.037),0.250 (0.046),37.095 (0.100)
cnn,0.586 (0.021),0.457 (0.022),36.650 (0.141)
unet,0.696 (0.036),0.784 (0.023),19.761 (0.721)


In [111]:
print(results_seg_f.to_latex(float_format="%.3f"))

\begin{tabular}{llll}
\toprule
{} &     train/dice &     valid/dice &      valid/hd95 \\
model    &                &                &                 \\
\midrule
CubeUNet &  0.360 (0.353) &  0.171 (0.213) &  38.633 (5.027) \\
FTUNet   &  0.422 (0.312) &  0.206 (0.178) &  38.266 (2.109) \\
IcoUNet  &  0.155 (0.110) &  0.035 (0.022) &  40.030 (0.026) \\
UNet     &  0.533 (0.037) &  0.250 (0.046) &  37.095 (0.100) \\
cnn      &  0.586 (0.021) &  0.457 (0.022) &  36.650 (0.141) \\
unet     &  0.696 (0.036) &  0.784 (0.023) &  19.761 (0.721) \\
\bottomrule
\end{tabular}



In [112]:
res

[['FTUNet', 0.5825098276138305, 0.31708497554063797, 37.35141227618557],
 ['FTUNet', 0.6206750353177388, 0.30030603542303047, 36.768343493048675],
 ['FTUNet', 0.06266666650772094, 3.469249249169769e-15, 40.676866343223885],
 ['CubeUNet', 0.7288077155749003, 0.5398469826517006, 28.575463605075193],
 ['CubeUNet', 0.6410477717717489, 0.2328113065159417, 41.55713788205686],
 ['CubeUNet', 0.6694541374842325, 0.23805783825616042, 39.25313779209166],
 ['UNet', 0.5622368176778157, 0.28670470702151457, 36.98285321714549],
 ['UNet', 0.54553329149882, 0.2646973212560018, 37.17669601938748],
 ['UNet', 0.490800541639328, 0.19755957008649905, 37.1246449548801],
 ['IcoUNet', 0.03877817938725154, 0.011045837549803157, 40.000889993008606],
 ['IcoUNet', 0.168288587530454, 0.03867291224499544, 40.044831852917646],
 ['IcoUNet', 0.257794052362442, 0.05534172475648423, 40.04539551657335],
 ['CubeUNet', 0.016948264837265015, 0.008970247255638242, 40.37132329297158],
 ['CubeUNet', 0.006668298846731583, 0.0022

In [10]:
runs = wandb.Api().runs(path='ASOCA_final', filters={
    'config.seed': {'$in': [0,11,42]},
    'config.model/model': {"$regex": "models.classification.*"}, 
    })


for i, run in enumerate(runs):
    if run.summary.get('test/acc') is not None: continue
    print(f'Predicting on {run.name}, {run.id} ({i+1}/{len(runs)})')

    model_params = { k.split('/')[-1]:v for k,v in run.config.items() if 'model' in k }
    class_name = model_params['model']
    del model_params['model']
    if 'initialize' in model_params: model_params['initialize'] = False

Predicting on worthy-hill-450, 1yuhrbr7 (22/36)
Predicting on colorful-planet-449, 2s9q6wk6 (23/36)
Predicting on fresh-sound-445, 3nc1oi7i (24/36)
Predicting on silvery-monkey-441, 125j9nx9 (25/36)
Predicting on true-vortex-437, rq7viz7o (26/36)
Predicting on likely-river-429, 17lmukgc (28/36)


In [2]:
model_dir = '/var/scratch/ebekkers/damyan/models/'

In [51]:
run = [x for x in list(runs) if x.name == 'worthy-hill-450' ][0]

In [52]:
run.files()[0]

<File ASOCA_final/1yuhrbr7/checkpoints/epoch=21-step=3453.ckpt () 38.0MiB>

In [53]:
import sys
sys.path.append('..')
from train import get_class
import pytorch_lightning as plt
import pickle as pkl
import torch

In [54]:
with run.files()[0].download(model_dir, replace=True) as model_f:
    model_params = { k.split('/')[-1]:v for k,v in run.config.items() if 'model' in k }
    class_name = model_params['model']
    del model_params['model']
    model: plt.LightningModule = get_class(class_name)(**model_params)

In [74]:
import wandb
api = wandb.Api()
run = api.run("/dkalev/ASOCA_final/runs/th3ogbe8")

print(run.history())

    gradients/decoders.2.decoder.bnorm1.regular_bias  \
0  {'values': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...   

               gradients/decoders.4.upsampler.1.bias  \
0  {'_type': 'histogram', 'bins': [-0.00105553364...   

          gradients/bottleneck.bnorm1.regular_weight  \
0  {'_type': 'histogram', 'packedBins': {'count':...   

            gradients/encoders.3.bnorm1.regular_bias  \
0  {'_type': 'histogram', 'packedBins': {'min': -...   

                  gradients/encoders.4.conv2.weights  \
0  {'_type': 'histogram', 'packedBins': {'min': -...   

                  gradients/encoders.0.conv1.weights  \
0  {'_type': 'histogram', 'packedBins': {'min': -...   

            gradients/encoders.1.bnorm1.regular_bias  \
0  {'_type': 'histogram', 'packedBins': {'min': -...   

  gradients/decoders.1.decoder.bnorm1.regular_weight  \
0  {'_type': 'histogram', 'packedBins': {'size': ...   

  gradients/decoders.3.decoder.bnorm2.regular_weight  \
0  {'_type': 'histogram', 'bins': [-0.05